In [54]:
import pandas as pd
import os
%xmode minimal
pd.set_option("mode.copy_on_write", True)

Exception reporting mode: Minimal


# 9. Tidy data
### 9.1. Billboard

In [55]:
billbo = pd.read_csv("../DsKi_DataPreparation/data/Aufgaben/billboard.csv")

In [56]:
billbo.head()

,year,artist,track,time,date.entered,wk1,wk2,wk3,wk4,wk5,...,wk67,wk68,wk69,wk70,wk71,wk72,wk73,wk74,wk75,wk76
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,87,82.0,72.0,77.0,87.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,91,87.0,92.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,81,70.0,68.0,67.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,3 Doors Down,Loser,4:24,2000-10-21,76,76.0,72.0,69.0,67.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,57,34.0,25.0,17.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
# Berechnen der Wochen in den Top 100
week_columns = [col for col in billbo.columns if col.startswith('wk')]
billbo['Weeks'] = billbo[week_columns].notna().sum(axis=1)

# Umformen des DataFrames in ein tidy format
tidy_billboard = billbo.melt(id_vars=["year", "artist", "track", "time", "date.entered", "Weeks"])

# Bereinigen der Woche und Umbenennen der Spalten
tidy_billboard.rename(columns={"date.entered": "date_entered"}, inplace=True)

# Optional: Berechnen der neuen Datumsspalte
tidy_billboard["date"] = pd.to_datetime(tidy_billboard["date_entered"]) + \
                         pd.to_timedelta(tidy_billboard["Weeks"] - 1, unit='w')

In [58]:
tidy_billboard = tidy_billboard[['year', 'artist', 'track', 'time', 'date_entered', 'Weeks', 'date']]


In [59]:
tidy_billboard.head()

,year,artist,track,time,date_entered,Weeks,date
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,7,2000-04-08
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,3,2000-09-16
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,53,2001-04-07
3,2000,3 Doors Down,Loser,4:24,2000-10-21,20,2001-03-03
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,18,2000-08-12


In [60]:
data = {
"name" : ["Philipp Woods", "Philipp Woods", "Philipp Woods",
"Jessica Cordero", "Jessica Cordero"],
"property" : ["age", "height", "age", "age", "height"],
"values" : [45, 186, 50, 37, 156]
}
df = pd.DataFrame(data)
df

,name,property,values
0,Philipp Woods,age,45
1,Philipp Woods,height,186
2,Philipp Woods,age,50
3,Jessica Cordero,age,37
4,Jessica Cordero,height,156


a) Formen Sie nach tidy data um. Welches Problem entsteht? <br>
b) Wie kann man das lösen? <br>
c) Wie könnte man eine Spalte ergänzen, so dass die Werte eindeutig identifiziert
werden?

In [65]:
# Schritt 2: Hinzufügen einer Spalte zur eindeutigen Identifizierung
df['id'] = df.groupby(['name', 'property']).cumcount()

# Schritt 3: Umformung unter Beibehaltung der Eindeutigkeit
tidy_df_unique = df.pivot_table(index=['name', 'id'], columns='property', values='values').reset_index()
tidy_df_unique.columns.name = None  # Entfernt den Namen der Spalten-Gruppe
tidy_df_unique


,name,id,age,height
0,Jessica Cordero,0,37.0,156.0
1,Philipp Woods,0,45.0,186.0
2,Philipp Woods,1,50.0,NaN


9.3 Wandeln Sie nach tidy data. Was sind die Variablen?

In [66]:
# Zuerst erstellen wir einen DataFrame basierend auf den Informationen aus dem Bild.
# Da wir den Inhalt des Bildes nicht direkt auslesen können, müssen wir die Daten manuell eingeben.
data = {
    "pregnant": ["yes", "no"],
    "male": [None, 20],
    "female": [10, 12]
}

# DataFrame erstellen
wide_df = pd.DataFrame(data)

# Umwandlung in Tidy Data Format
tidy_df = wide_df.melt(id_vars=["pregnant"], value_vars=["male", "female"], 
                       var_name="gender", value_name="count")
tidy_df

,pregnant,gender,count
0,yes,male,NaN
1,no,male,20.0
2,yes,female,10.0
3,no,female,12.0


Beispiel Tidy Data

In [68]:
data = {
    "Stadt": ["Berlin", "Hamburg"],
    "Tag1": [20, 18],
    "Tag2": [22, 19]
}
df = pd.DataFrame(data)

# Umwandeln in Tidy Data
tidy_df = df.melt(id_vars=["Stadt"], value_vars=["Tag1", "Tag2"],
                  var_name="Tag", value_name="Temperatur")

tidy_df

,Stadt,Tag,Temperatur
0,Berlin,Tag1,20
1,Hamburg,Tag1,18
2,Berlin,Tag2,22
3,Hamburg,Tag2,19


In [70]:
data = {
    "Produkt": ["Apfel", "Banane"],
    "Q1": [300, 200],
    "Q2": [250, 220]
}
df = pd.DataFrame(data)

# Umwandeln in Tidy Data
tidy_df = df.melt(id_vars=["Produkt"], value_vars=["Q1", "Q2"],
                  var_name="Quartal", value_name="Verkaufszahlen")
tidy_df


,Produkt,Quartal,Verkaufszahlen
0,Apfel,Q1,300
1,Banane,Q1,200
2,Apfel,Q2,250
3,Banane,Q2,220
